### Simple database queries

In [1]:
import sqlite3
import pandas as pd

In [40]:
# Load Customers
df = pd.read_csv('../../datasets/archive/olist_customers_dataset.csv')
df.columns = df.columns.str.strip()
con = sqlite3.connect("data/olist.db")
df.to_sql("customers", con)
con.close()

In [41]:
# Load Orders
df = pd.read_csv('../../datasets/archive/olist_orders_dataset.csv')
df.columns = df.columns.str.strip()
con = sqlite3.connect("data/olist.db")
df.to_sql("orders", con)
con.close()

In [42]:
# Load Products
df = pd.read_csv('../../datasets/archive/olist_order_reviews_dataset.csv')
df.columns = df.columns.str.strip()
con = sqlite3.connect("data/olist.db")
df.to_sql("order_reviews", con)
con.close()

In [2]:
from langchain import OpenAI, SQLDatabase, SQLDatabaseChain

In [4]:
db = SQLDatabase.from_uri("sqlite:///data/olist.db")
# llm = OpenAI(temperature=0, verbose=True)

In [5]:
from langchain.llms import VertexAI
llm = VertexAI(temperature=0)

In [6]:
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

In [7]:
db_chain.run("""What column names we have in orders?""")



> Entering new SQLDatabaseChain chain...
What column names we have in orders?
SQLQuery:SELECT * FROM orders LIMIT 0
SQLResult: []
Answer:index, order_id, customer_id, order_status, order_purchase_timestamp, order_approved_at, order_delivered_carrier_date, order_delivered_customer_date, order_estimated_delivery_date
> Finished chain.


'index, order_id, customer_id, order_status, order_purchase_timestamp, order_approved_at, order_delivered_carrier_date, order_delivered_customer_date, order_estimated_delivery_date'

In [9]:
db_chain.run("""What is the average delivery time for the orders, for each state? Do not limit results.""")



> Entering new SQLDatabaseChain chain...
What is the average delivery time for the orders, for each state? Do not limit results.
SQLQuery:SELECT customer_state, AVG(CAST(strftime('%s', order_delivered_customer_date) - strftime('%s', order_approved_at) AS NUMERIC)) AS average_delivery_time FROM orders GROUP BY customer_state

OperationalError: (sqlite3.OperationalError) no such column: customer_state
[SQL: SELECT customer_state, AVG(CAST(strftime('%s', order_delivered_customer_date) - strftime('%s', order_approved_at) AS NUMERIC)) AS average_delivery_time FROM orders GROUP BY customer_state]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [10]:
db_chain.run("""
What is the product with the most review comments?
""")



> Entering new SQLDatabaseChain chain...

What is the product with the most review comments?

SQLQuery:SELECT product_name FROM products ORDER BY review_count DESC LIMIT 1

OperationalError: (sqlite3.OperationalError) no such table: products
[SQL: SELECT product_name FROM products ORDER BY review_count DESC LIMIT 1]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [11]:
db_chain.run("""
Retrieve 40 samples order reviews that have comment messages and a review score less than 3. 
What are the main complains?
""")



> Entering new SQLDatabaseChain chain...

Retrieve 40 samples order reviews that have comment messages and a review score less than 3. 
What are the main complains?

SQLQuery:SELECT review_comment_message FROM order_reviews WHERE review_score < 3 AND review_comment_message IS NOT NULL LIMIT 40
SQLResult: [('GOSTARIA DE SABER O QUE HOUVE, SEMPRE RECEBI E ESSA COMPRA AGORA ME DECPCIONOU',), ('Péssimo',), ('Não gostei ! Comprei gato por lebre',), ('Sempre compro pela Internet e a entrega ocorre antes do prazo combinado, que acredito ser o prazo máximo. No stark o prazo máximo já se esgotou e ainda não recebi o produto.',), ('Nada de chegar o meu pedido.',), ('recebi somente 1 controle Midea Split ESTILO.\r\nFaltou Controle Remoto para Ar Condicionado Consul',), ('O produto não chegou no prazo estipulado e causou transtorno, pq programei a viagem de férias do meu filho, baseado no prazo. Moro na Bahia e ele em Cuiabá sozinho. Agora, a casa está vazia. ',), ('Produto muito inferior, mal a

''